In [2]:
import numpy as np

## Maximum SubArray 
Find a non-empty, contiguous subarray of A whose values have the largest sum. In this problem, the array usually contains negative numbers. If not, then the maximum subarray would simply be the whole array.         

  



### Brute Force It
Cost $O(n^2)$

In [23]:
Arr = [-2, 1, -3, 4, -1, 2, 1, -5, 4]

def get_a_hammer(A):# Get A Hammer
    #left_ind, right_ind, sum
    max_sum = [0, 0, 0]
    len_arr = len(A)
    for i in range(len_arr):
        tmp_sum = 0
        for j in range(i, len_arr):
            tmp_sum += A[j]
            if tmp_sum > max_sum[2]:
                max_sum = [i, j, tmp_sum]
    
    return max_sum

In [22]:
get_a_hammer(Arr)

[3, 6, 6]

### Recursive Procedure
Observe that for array $A[low,high]$ and midpoint $mid$, a contigous subarray $A[i,j]$ exists $\ni$:
1. it exists entirely in $A[low, mid]$, $low \leq i \leq j \leq mid$
2. it exists entirely in $A[mid, high]$, $mid \leq i \leq j \leq high$
3. it crosses $mid$, $low \leq i \leq mid < j \leq high$ 

Idea: For an array, find the max subarray of the left half, of the right half, and for the case when it includes the midpoint. Choose the max of the three. Most of the real computation is done in the computation of the max subarray that crossed the midpoint. The left and right max subarrays are just computed through recursive comparison.   

Procedure:
1. Recursively split array, with base case returning the element of an array of length 1
2. Compute max subarray that straddles midpiont
3. Return the max of the 3 

Example: 
1. For array [-2, 3, 1, 5]
    1. split into left [-2, 3] $\rightarrow$ return [3]
        1. split into left [-2] $\rightarrow$ return max left subarray [-2]
        2. split into right [3] $\rightarrow$ return max right subarray [3]
        3. Compute max subarray of [-2, 3] that crosses midpoint $\rightarrow$ return [-2, 3] (sum = 1)
    2. split into right [1, 5] $\rightarrow$ [1, 5] (sum = 6)
        1. split into left [1] $\rightarrow$ return max left subarray [1]
        2. split into right [5] $\rightarrow$ return max right subarray [5]
        3. Compute max subarray of [1, 5] that crosses midpoint $\rightarrow$ return [1, 5] (sum = 6)
    3. Compute max subarray of [-2, 3, 1, 5] that crosses midpoint $\rightarrow$ returns [3, 1, 5] (sum = 9)
2. Returns [3, 1, 5] (sum = 9)


#### Max Crossing Subarray
Finds the maximum subarray of A such that sum($A[i, j]$) is maximixed $ \forall i,j \ni low \leq i \leq mid < j \leq high$


In [29]:
def find_max_crossing_subarray(A, low, mid, high):
    ''' Given inputs orignal array A, low ind low, high ind high, and mid ind mid
        returns left and right indices of max subarray that crosses midpoint and its sum.
        Expects midpoint to be "low" when even, e.g. [a, b, c, d] -> midpoint is 1 not 2
    '''
    # find max subbarray left
    max_left_sum = -1*np.inf
    sum_ = 0
    max_left_ind = mid
    
    for i in range(mid, low-1, -1):
        sum_ += A[i]
        if sum_ > max_left_sum:
            max_left_sum = sum_
            max_left_ind = i

    # find max subbarray right
    max_right_sum = -1*np.inf
    sum_ = 0
    max_right_ind = mid+1
    
    for i in range(mid+1, high+1):
        sum_ += A[i]
        if sum_ > max_right_sum:
            max_right_sum = sum_
            max_right_ind = i

    return max_left_ind, max_right_ind, max_left_sum+max_right_sum
            

### Recursive Find Max Subarray
Cost is $O(nlog(n))$

In [32]:
def get_max_subarray(A, low, high):
    if low == high:
        #only one element left
        return low, high, A[low]
    
    mid = (high+low) // 2
    left_low, left_high, left_sum = get_max_subarray(A, low, mid)

    right_low, right_high, right_sum = get_max_subarray(A, mid+1, high)

    cross_low,cross_high, cross_sum = find_max_crossing_subarray(A, low, mid, high)


    if left_sum >= right_sum and left_sum >= cross_sum:
        return left_low, left_high, left_sum
    elif right_sum >= left_sum and right_sum >= cross_sum:
        return right_low, right_high, right_sum
    else:
        return cross_low, cross_high, cross_sum


In [46]:
A = [-2, 1, -3, 4, -1, 2, 1, -5, 4]
max_subarray_out = get_max_subarray(A, 0, len(A)-1)
print(f"Bounds A[{max_subarray_out[0]}, {max_subarray_out[1]}], SubArray: {A[max_subarray_out[0]:max_subarray_out[1]+1]}, Sum: {max_subarray_out[2]}")

Bounds A[3, 6], SubArray: [4, -1, 2, 1], Sum: 6


### Kadane's Algorithm

Let the $M_{k}$ denote the maximum subarry of $A$ ending at index $k$. Then the maximum subarray at point $n$ is either $A[n]$ or $M_{n-1} + A[n]$. Based upon this, can produce an optimal algorithm.   
Cost $O(n)$

In [50]:
def kadane_max_subarray(A):
    running_max = global_max = A[0]
    left = right = 0

    for i in range(1, len(A)):
        if running_max + A[i] < A[i]:
            left = i
            running_max = A[i]
        else:
            running_max = running_max + A[i]
            
        if running_max > global_max:
            global_max = running_max
            right = i
            
    return left, right, global_max

In [53]:
max_subarray_out = kadane_max_subarray(A)
print(f"Bounds A[{max_subarray_out[0]}, {max_subarray_out[1]}], SubArray: {A[max_subarray_out[0]:max_subarray_out[1]+1]}, Sum: {max_subarray_out[2]}")

Bounds A[3, 6], SubArray: [4, -1, 2, 1], Sum: 6


In [54]:
A2 = [-3, -4, 5, -1, 2, -4, 6, -1]
max_subarray_out = kadane_max_subarray(A2)
print(f"Bounds A[{max_subarray_out[0]}, {max_subarray_out[1]}], SubArray: {A[max_subarray_out[0]:max_subarray_out[1]+1]}, Sum: {max_subarray_out[2]}")

Bounds A[2, 6], SubArray: [-3, 4, -1, 2, 1], Sum: 8
